# Apache Spark Streaming: Procesamiento en Tiempo Real

## Objetivos de Aprendizaje
- Dominar Spark Structured Streaming
- Implementar procesamiento de ventanas y agregaciones
- Integrar con Kafka y otras fuentes de streaming
- Manejar estado y checkpointing
- Optimizar rendimiento en streaming

## Requisitos
- PySpark 3.x
- Python 3.8+
- Kafka (opcional)
- Delta Lake (opcional)

In [ ]:
# Instalación de dependencias
import sys
!{sys.executable} -m pip install pyspark pandas numpy -q

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, window, count, sum as spark_sum, avg, max as spark_max,
    current_timestamp, to_json, from_json, struct, expr
)
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, 
    DoubleType, TimestampType
)
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

print("Librerías importadas correctamente")

## 1. Inicializar Spark Session

In [ ]:
# Crear Spark Session con configuración para Streaming
spark = SparkSession.builder \
    .appName("SparkStreamingAdvanced") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.sql.streaming.schemaInference", "true") \
    .config("spark.streaming.stopGracefullyOnShutdown", "true") \
    .getOrCreate()

# Configurar nivel de logging
spark.sparkContext.setLogLevel("WARN")

print(f"Spark Version: {spark.version}")
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")

## 2. Definir Esquemas para Streaming

In [ ]:
# Esquema para eventos de e-commerce
ecommerce_schema = StructType([
    StructField("event_id", StringType(), False),
    StructField("timestamp", TimestampType(), False),
    StructField("user_id", StringType(), False),
    StructField("event_type", StringType(), False),
    StructField("product_id", StringType(), False),
    StructField("product_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("quantity", IntegerType(), True)
])

# Esquema para logs de aplicación
log_schema = StructType([
    StructField("timestamp", TimestampType(), False),
    StructField("level", StringType(), False),
    StructField("service", StringType(), False),
    StructField("message", StringType(), True),
    StructField("error_code", IntegerType(), True),
    StructField("user_id", StringType(), True)
])

print("Esquemas definidos")
print("\nEsquema E-commerce:")
print(ecommerce_schema)

## 3. Simulación de Fuente de Streaming

In [ ]:
# Crear datos de ejemplo para simular streaming
def generate_sample_data(n_records=1000):
    """
    Generar datos de muestra para streaming
    """
    np.random.seed(42)
    
    base_time = datetime.now()
    
    data = []
    for i in range(n_records):
        event = {
            'event_id': f'evt_{i:06d}',
            'timestamp': base_time + timedelta(seconds=i),
            'user_id': f'user_{np.random.randint(1, 101)}',
            'event_type': np.random.choice(['view', 'add_to_cart', 'purchase', 'remove'], p=[0.5, 0.25, 0.15, 0.1]),
            'product_id': f'prod_{np.random.randint(1, 51)}',
            'product_name': np.random.choice(['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Headphones']),
            'category': np.random.choice(['Electronics', 'Accessories', 'Computers']),
            'price': round(np.random.uniform(10, 2000), 2),
            'quantity': np.random.randint(1, 5)
        }
        data.append(event)
    
    return data


# Generar datos
sample_data = generate_sample_data(1000)
df_sample = spark.createDataFrame(sample_data, schema=ecommerce_schema)

print(f"Generados {df_sample.count()} registros de muestra")
df_sample.show(5, truncate=False)

## 4. Streaming con Rate Source (Simulación)

In [ ]:
# Crear stream simulado con rate source
rate_stream = spark.readStream \
    .format("rate") \
    .option("rowsPerSecond", 10) \
    .option("numPartitions", 2) \
    .load()

# Enriquecer con datos simulados
from pyspark.sql.functions import rand, when, lit

enriched_stream = rate_stream \
    .withColumn("user_id", (rand() * 100).cast("int")) \
    .withColumn("event_type", 
        when(rand() < 0.5, "view")
        .when(rand() < 0.75, "add_to_cart")
        .when(rand() < 0.9, "purchase")
        .otherwise("remove")
    ) \
    .withColumn("product_id", (rand() * 50).cast("int")) \
    .withColumn("price", (rand() * 1990 + 10).cast("double")) \
    .withColumn("quantity", (rand() * 4 + 1).cast("int"))

print("Stream enriquecido creado")
print("Schema:")
enriched_stream.printSchema()

## 5. Agregaciones por Ventanas de Tiempo

In [ ]:
# Agregaciones por ventana de tiempo
windowed_counts = enriched_stream \
    .withWatermark("timestamp", "10 seconds") \
    .groupBy(
        window(col("timestamp"), "30 seconds", "10 seconds"),
        col("event_type")
    ) \
    .agg(
        count("*").alias("event_count"),
        spark_sum(expr("price * quantity")).alias("total_revenue"),
        avg("price").alias("avg_price")
    ) \
    .orderBy("window")

print("Query de agregación por ventanas configurado")

## 6. Procesamiento Stateful

In [ ]:
# Mantener estado acumulado por usuario
user_aggregations = enriched_stream \
    .groupBy("user_id") \
    .agg(
        count("*").alias("total_events"),
        spark_sum(when(col("event_type") == "purchase", 1).otherwise(0)).alias("purchases"),
        spark_sum(when(col("event_type") == "purchase", 
                      expr("price * quantity")).otherwise(0)).alias("total_spent"),
        spark_max("timestamp").alias("last_activity")
    )

print("Query de agregación por usuario configurado")

## 7. Detección de Patrones en Tiempo Real

In [ ]:
# Detectar usuarios con comportamiento anómalo
anomaly_detection = enriched_stream \
    .withWatermark("timestamp", "5 minutes") \
    .groupBy(
        window(col("timestamp"), "1 minute"),
        col("user_id")
    ) \
    .agg(
        count("*").alias("events_per_minute"),
        spark_sum(when(col("event_type") == "purchase", 
                      expr("price * quantity")).otherwise(0)).alias("spend_per_minute")
    ) \
    .filter(
        (col("events_per_minute") > 50) |  # Más de 50 eventos por minuto
        (col("spend_per_minute") > 10000)   # Más de $10,000 por minuto
    )

print("Query de detección de anomalías configurado")

## 8. Ejemplo de Query con Output Completo

In [ ]:
# Mostrar resultados en consola (modo batch para demo)
# NOTA: En producción usarías .writeStream() en lugar de show()

print("\n=== ANÁLISIS BATCH DE DATOS DE MUESTRA ===")
print("\n1. Eventos por tipo:")
df_sample.groupBy("event_type").count().orderBy("count", ascending=False).show()

print("\n2. Revenue por categoría:")
df_sample.filter(col("event_type") == "purchase") \
    .groupBy("category") \
    .agg(
        count("*").alias("num_purchases"),
        spark_sum(expr("price * quantity")).alias("total_revenue"),
        avg("price").alias("avg_price")
    ) \
    .orderBy("total_revenue", ascending=False) \
    .show()

print("\n3. Top 10 usuarios más activos:")
df_sample.groupBy("user_id") \
    .agg(
        count("*").alias("total_events"),
        spark_sum(when(col("event_type") == "purchase", 1).otherwise(0)).alias("purchases")
    ) \
    .orderBy("total_events", ascending=False) \
    .limit(10) \
    .show()

print("\n4. Tasa de conversión por producto:")
conversion_rate = df_sample.groupBy("product_name") \
    .agg(
        count("*").alias("total_events"),
        spark_sum(when(col("event_type") == "view", 1).otherwise(0)).alias("views"),
        spark_sum(when(col("event_type") == "purchase", 1).otherwise(0)).alias("purchases")
    )

conversion_rate.withColumn(
    "conversion_rate",
    (col("purchases") / col("views") * 100)
).orderBy("conversion_rate", ascending=False).show()

## 9. Escribir Stream a Diferentes Sinks

In [ ]:
# Ejemplo de configuración de escritura (comentado para evitar ejecución)

# 1. Escribir a consola (desarrollo/debug)
console_query_config = {
    'outputMode': 'complete',  # complete, append, update
    'format': 'console',
    'trigger': {'processingTime': '10 seconds'},
    'options': {
        'truncate': False,
        'numRows': 20
    }
}

# 2. Escribir a Parquet (data lake)
parquet_query_config = {
    'outputMode': 'append',
    'format': 'parquet',
    'path': '/path/to/output',
    'checkpointLocation': '/path/to/checkpoint',
    'trigger': {'processingTime': '1 minute'},
    'options': {
        'compression': 'snappy',
        'partitionBy': 'date'
    }
}

# 3. Escribir a Kafka
kafka_query_config = {
    'outputMode': 'append',
    'format': 'kafka',
    'options': {
        'kafka.bootstrap.servers': 'localhost:9092',
        'topic': 'processed-events',
        'checkpointLocation': '/path/to/checkpoint'
    }
}

# 4. Escribir a Delta Lake
delta_query_config = {
    'outputMode': 'append',
    'format': 'delta',
    'path': '/path/to/delta',
    'checkpointLocation': '/path/to/checkpoint',
    'options': {
        'mergeSchema': True,
        'optimizeWrite': True
    }
}

print("Configuraciones de sink definidas (ver código para detalles)")

## 10. Métricas y Monitoreo

In [ ]:
# Función para monitorear estado del stream
def monitor_stream_metrics(query):
    """
    Extraer métricas del streaming query
    """
    status = query.status
    
    metrics = {
        'isDataAvailable': status['isDataAvailable'],
        'isTriggerActive': status['isTriggerActive'],
        'message': status['message']
    }
    
    if 'inputRowsPerSecond' in status:
        metrics['inputRowsPerSecond'] = status['inputRowsPerSecond']
    
    if 'processedRowsPerSecond' in status:
        metrics['processedRowsPerSecond'] = status['processedRowsPerSecond']
    
    return metrics


# Ejemplo de métricas a monitorear
print("\n=== MÉTRICAS CLAVE PARA MONITOREO ===")
print("""
1. Input Rate: Eventos por segundo recibidos
2. Processing Rate: Eventos por segundo procesados
3. Batch Duration: Tiempo de procesamiento por batch
4. Trigger Interval: Intervalo entre ejecuciones
5. Watermark: Retraso máximo aceptado
6. Estado del Query: Activo, inactivo, error
7. Checkpoint Location: Para recuperación de fallos
""")

## 11. Optimización de Rendimiento

In [ ]:
# Configuraciones de optimización
optimization_configs = {
    # Particionamiento
    'spark.sql.shuffle.partitions': '200',  # Número de particiones para shuffles
    'spark.default.parallelism': '200',     # Paralelismo por defecto
    
    # Memoria
    'spark.executor.memory': '4g',
    'spark.driver.memory': '2g',
    'spark.memory.fraction': '0.8',
    
    # Streaming específico
    'spark.sql.streaming.minBatchesToRetain': '100',
    'spark.sql.streaming.stateStore.providerClass': 'org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider',
    
    # Optimización de escritura
    'spark.sql.adaptive.enabled': 'true',
    'spark.sql.adaptive.coalescePartitions.enabled': 'true',
}

print("\n=== MEJORES PRÁCTICAS DE OPTIMIZACIÓN ===")
print("""
1. Usar watermarks para limpiar estado antiguo
2. Particionar datos por columnas clave
3. Configurar apropiadamente spark.sql.shuffle.partitions
4. Usar triggers basados en tiempo para controlar frecuencia
5. Implementar checkpointing para recuperación
6. Monitorear métricas constantemente
7. Usar Delta Lake para ACID transactions
8. Implementar compactación de archivos pequeños
9. Optimizar esquemas y evitar tipos genéricos
10. Considerar micro-batching vs continuous processing
""")

## Resumen y Arquitectura Enterprise

### Arquitectura Típica de Streaming:
```
Fuentes de Datos       Ingesta           Procesamiento        Almacenamiento        Consumo
────────────────       ───────           ──────────────       ──────────────        ───────
Kafka/Kinesis    →    Spark       →     Transformaciones  →   Delta Lake      →    BI Tools
IoT Devices      →    Streaming   →     Agregaciones      →   Data Lake       →    ML Models
APIs             →                →     Joins             →   Warehouse       →    Dashboards
Logs             →                →     Windows           →   Cache (Redis)   →    Alertas
```

### Patrones Avanzados:

#### 1. Lambda Architecture
- **Batch Layer**: Procesamiento histórico completo
- **Speed Layer**: Procesamiento en tiempo real
- **Serving Layer**: Combina ambas vistas

#### 2. Kappa Architecture
- Solo capa de streaming
- Todo procesamiento en tiempo real
- Reprocesamiento desde el inicio del stream

#### 3. Delta Architecture
- Basada en Delta Lake
- ACID transactions
- Time travel
- Schema evolution

### Casos de Uso Enterprise:

1. **Detección de Fraude en Tiempo Real**
   - Análisis de patrones sospechosos
   - Machine Learning en streaming
   - Alertas automáticas

2. **Recomendaciones Personalizadas**
   - Seguimiento de comportamiento en tiempo real
   - Actualización de perfiles de usuario
   - A/B testing dinámico

3. **Monitoreo de Infraestructura**
   - Logs y métricas en tiempo real
   - Detección de anomalías
   - Auto-scaling basado en carga

4. **IoT y Telemetría**
   - Procesamiento de sensores
   - Mantenimiento predictivo
   - Optimización de operaciones

### Consideraciones de Producción:

- **Alta Disponibilidad**: Cluster mode, múltiples workers
- **Fault Tolerance**: Checkpointing, Write-Ahead Logs
- **Escalabilidad**: Auto-scaling, dynamic allocation
- **Seguridad**: Kerberos, SSL/TLS, encryption at rest
- **Monitoreo**: Prometheus, Grafana, CloudWatch
- **Testing**: Unit tests, integration tests, chaos engineering

### Recursos Adicionales:
- [Spark Structured Streaming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Delta Lake Documentation](https://docs.delta.io/latest/index.html)
- [Databricks Streaming Best Practices](https://docs.databricks.com/structured-streaming/index.html)

In [ ]:
# Limpiar recursos
spark.stop()
print("Spark Session cerrada")